In [105]:
import pandas as pd
import matplotlib
#matplotlib.use('Agg')
import os
from time import time
import numpy as np
import pylab as pl
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn import model_selection
import itertools
import shutil
from sklearn import preprocessing


from  sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer

In [106]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

In [107]:
df = pd.read_excel('processed_batch (3).xlsx', sheet_name='Sheet1')

In [108]:
df

,Reviewer,Review date,Description,Rating,OPINION_UNIT,Classification,Confidence,Classification.1,Confidence.1
0,Shuttitt Uppitt,"February 2, 2020","(3.2.7) It works fine. It's smart to put the ""...",Rated 5 stars out of five stars,(3.2.7) It works fine.,Ease of Use,0.465,Positive,0.957
1,NaN,NaN,NaN,NaN,"It's smart to put the ""Delete"" option at the B...",Features,0.427,Neutral,0.539
2,NaN,NaN,NaN,NaN,(At other users.,Ease of Use,0.493,Neutral,0.756
3,NaN,NaN,NaN,NaN,Use FILE MANAGERS to delete videos/ images/ mu...,Ease of Use,0.522,Neutral,0.351
4,NaN,NaN,NaN,NaN,Now I hope to see better performance for non S...,Features,0.684,Negative,0.818
...,...,...,...,...,...,...,...,...,...
248,NaN,NaN,NaN,NaN,Get the COMPLETE PACKAGE with VLC Player.,NaN,NaN,Neutral,0.682
249,NaN,NaN,NaN,NaN,"When new codecs become available, they will be...",Features,0.618,Neutral,0.699
250,NaN,NaN,NaN,NaN,I haven't used another video player in YEARS!,Features:Ease of Use,0.532:0.458,Positive,0.786
251,Terence 1Plus,"February 5, 2020",,Rated 5 stars out of five stars,NaN,NaN,NaN,Positive,0.984


In [109]:
# data = []
# tag = []
# for i in range(df['OPINION_UNIT'].shape[0]):
#     if df['OPINION_UNIT'][i]!=np.isnan():
#         data.append(df['OPINION_UNIT'][i])
#         tag.append(df['Classification'][i])

In [110]:
d = pd.read_csv('dataset_with_labels.csv')

In [111]:
d

,Name,Review,Label,UpVotes,Rating,Date,Keywords
0,Shuttitt Uppitt,(3.2.7) It works fine. It's smart to put the D...,"Feature Request,User Interface,Compatibility I...",417.0,4.4,"February 2, 2020","['works fine', 'smart', 'put', 'delete option'..."
1,Nate,Seek track is impossible to use. Whether I hol...,Functional Complaint,627.0,5.0,"February 4, 2020","['seek track', 'impossible', 'hold', 'fast for..."
2,Sir Willard,I've been using the VLC Windows application fo...,"Update Issue,User Interface",97.0,2.0,"February 4, 2020","['ve', 'vlc windows application', 'years &', '..."
3,Chris L,The updates just make it worse. I used to like...,"User Interface,Update Issue,Feature Request",210.0,2.0,"February 10, 2020","['updates', 'make', 'worse', 'sorting', 'date'..."
4,Peter Blaikie,I love VLC on my PC. Used to like it on my And...,"Compatibility Issue,Functional Complaint",22.0,1.0,"February 14, 2020","['love vlc', 'pc', 'android phone', 'doesn', '..."
...,...,...,...,...,...,...,...
835,Jonathan Veith,all of a sudden playlist view its stuck on thu...,NaN,1.0,5.0,"December 11, 2019","['sudden playlist view', 'stuck', 'thumbnails'..."
836,Big Fat Frosty,No networks try again for few minutes on opens...,NaN,2.0,2.0,"December 10, 2019","['networks', 'minutes', 'opensubtitles', 'sear..."
837,Howard Denardo,Work fine for all files. I can stream files fr...,NaN,NaN,1.0,"December 31, 2019","['work fine', 'files', 'stream files', 'cloud'..."
838,Daniel Branco,New version is bad I try to stop video by doub...,NaN,NaN,3.0,"December 23, 2019","['version', 'bad', 'stop video', 'double tappi..."


In [113]:
# train_size = int(len(df) * .7)
train_posts = d['Review'][:90]
train_tags = d['Label'][:90]

test_posts = d['Review'][:12]
test_tags = d['Label'][:12]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(d['Review']) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(d['Label'][:102])
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)
# print(y_test)
num_classes = np.max(y_train) + 1
uniques, ids = np.unique(train_tags, return_inverse=True)
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)
# print(y_test)
batch_size = 32
epochs =8

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


Train on 81 samples, validate on 9 samples
Epoch 1/8
81/81 [==============================] - 3s 36ms/step - loss: 2.9234 - acc: 0.0617 - val_loss: 2.8745 - val_acc: 0.1111
Epoch 2/8
81/81 [==============================] - 0s 733us/step - loss: 2.5438 - acc: 0.3951 - val_loss: 2.6291 - val_acc: 0.4444
Epoch 3/8
81/81 [==============================] - 0s 682us/step - loss: 2.0945 - acc: 0.6049 - val_loss: 2.4200 - val_acc: 0.4444
Epoch 4/8
81/81 [==============================] - 0s 770us/step - loss: 1.7500 - acc: 0.6543 - val_loss: 2.2603 - val_acc: 0.4444
Epoch 5/8
81/81 [==============================] - 0s 688us/step - loss: 1.4739 - acc: 0.6667 - val_loss: 2.1571 - val_acc: 0.4444
Epoch 6/8
81/81 [==============================] - 0s 800us/step - loss: 1.2461 - acc: 0.7037 - val_loss: 2.1033 - val_acc: 0.4444
Epoch 7/8
81/81 [==============================] - 0s 721us/step - loss: 1.0412 - acc: 0.7284 - val_loss: 2.0824 - val_acc: 0.4444
Epoch 8/8
81/81 [========================

In [114]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

12/12 [==============================] - 0s 1ms/step
Test accuracy: 0.75


In [115]:
print(y_test[0])


[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [116]:
final_test = d['Review'][103:]

In [117]:
y_pred = model.predict(tokenize.texts_to_matrix(final_test))

In [118]:
y_classes = [np.argmax(y, axis=None, out=None) for y in y_pred]

In [119]:
y_classes = encoder.inverse_transform(y_classes)
# print(y_classes)